### Aprendizado Ensemble - Florestas Aleatórias

In [1]:
# bibliotecas utilizadas no desenvolvimento do projeto
from collections import Counter, defaultdict, OrderedDict
from functools import partial
import math, random
import pandas as pd
from csv import reader
import numpy as np
from random import seed
from random import randrange
from graphviz import Digraph

In [2]:
def build_dataset(path_file): 
    dataset = list()
    with open(path_file, 'r') as file:
        csv_reader = reader(file)
        rows = [r for r in csv_reader]
    # capturando os cabeçalhos
    headers = rows[0]
    
    remove_row_with_values_missing(rows, headers)
    
    for row in rows[1:len(rows)]:
        out_dic = {}
        for header_index in range(len(headers)):
            out_dic.update({ headers[header_index]: row[header_index]})
        
        # captura a classe
        class_key = list(out_dic.items())[-1]
        # remove a classe captura do dicionario
        del out_dic[list(out_dic.keys())[-1]]
        # montando o dataset de saída
        data_tuple = (out_dic, class_key[1])
        dataset.append(data_tuple)
    return dataset

# TODO: devemos remover linhas com dados faltando?
def remove_row_with_values_missing(rows, headers):
    quant = 0
    for row in rows[1:len(rows)]:
        for header_index in range(len(headers)):
            if(row[header_index] is None or row[header_index] == ""):
                quant = quant + 1
    

# construindo a base de dados a partir do arquivo fornecido
dataset = build_dataset('datasets/benchmark.csv')

1. Desenvolvimento do algoritmo de indução de uma árvore de decisão, usando como critério de seleção de atributos para divisão de nós o **Ganho de Informação (baseado no conceito de entropia)**. **Tratando tanto atributos categóricos quanto numéricos.**

In [3]:

def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum( data_entropy(subset) * len(subset) / total_count for subset in subsets )

def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups

def partition_by(inputs, attribute):
    """returns a dict of inputs partitioned by the attribute each input is a pair (attribute_dict, label)"""
    return group_by(inputs, lambda x: x[0][attribute])

def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values()) # retorna a media dos sub-conjuntos resultantes

def gain(class_entropy, mean_entropy):
    return class_entropy - mean_entropy;

def class_entropy(inputs):
    """retorna a entropia sobre a classe"""
    class_values = []
    for item in inputs:
        class_values.append(item[1]) # suponto a definição atual dos dados!
    probabilities = class_probabilities(class_values)
    class_entropy = entropy(probabilities)
    return class_entropy

def diff_class(inputs):
    classes = []
    for current_class in list(set([label for item, label in inputs if label])):
        classes.append({'class': current_class, 'amount': 0})
    for current_class in [label for item, label in inputs if label]:
        for my_class in classes:
            if(my_class['class']  == current_class):
                my_class['amount'] = my_class['amount'] + 1
    return classes

def highest_class(classes):
    my_class = classes[0]
    for current_class in classes:
        if(current_class['amount'] > my_class['amount']):
            my_class = current_class
    return my_class
    

def build_tree(inputs, split_candidates=None):

    # if this is our first pass,
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
              
    # obtendo as classes
    # [(classe, quantidade)]
    classes = diff_class(inputs)
    print(classes)
    print(highest_class(classes))
    
    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues
    
    # verificando a homogeneidade do cojunto
    if(len(classes) == 1):
        return classes[0]['class']

    if not split_candidates:          # if no split candidates left
        return highest_class(classes) # return the majority leaf

    # otherwise, split on the best attribute
    
    best_gain = 0
    best_attribute = None
    for candidate in split_candidates:
        mean_entropy = partition_entropy_by(inputs, candidate)
        print('[INFO] Atributo: ', candidate, '- entropia média: ', mean_entropy)
        gain_result = gain(class_entropy(inputs), mean_entropy)
        print('[INFO] Atributo: ', candidate, '- ganho: ', gain_result)
        if(gain_result > best_gain):
            best_gain = gain(class_entropy(inputs), mean_entropy)
            best_attribute = candidate
        print('--')
    
    print('----------------------------------------')
    print('[INFO] best_attribute: ', best_attribute)
    print('----------------------------------------')
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    # recursively build the subtrees
    subtrees = { attribute : build_tree(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    #subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

if __name__ == "__main__":
    # construindo a árvore sobre os dados construindos anteriormente
    tree = build_tree(dataset)
    print('\n')
    print('----------------------------------------')
    print("[INFO] Árvore construida:")
    print('----------------------------------------')
    print(tree)

[{'class': 'Sim', 'amount': 9}, {'class': 'Nao', 'amount': 5}]
{'class': 'Sim', 'amount': 9}
[INFO] Atributo:  Tempo - entropia média:  0.6935361388961919
[INFO] Atributo:  Tempo - ganho:  0.246749819774439
--
[INFO] Atributo:  Temperatura - entropia média:  0.9110633930116763
[INFO] Atributo:  Temperatura - ganho:  0.029222565658954647
--
[INFO] Atributo:  Umidade - entropia média:  0.7884504573082896
[INFO] Atributo:  Umidade - ganho:  0.15183550136234136
--
[INFO] Atributo:  Ventoso - entropia média:  0.8921589282623617
[INFO] Atributo:  Ventoso - ganho:  0.04812703040826927
--
----------------------------------------
[INFO] best_attribute:  Tempo
----------------------------------------
[{'class': 'Sim', 'amount': 2}, {'class': 'Nao', 'amount': 3}]
{'class': 'Nao', 'amount': 3}
[INFO] Atributo:  Temperatura - entropia média:  0.4
[INFO] Atributo:  Temperatura - ganho:  0.5709505944546686
--
[INFO] Atributo:  Umidade - entropia média:  0.0
[INFO] Atributo:  Umidade - ganho:  0.97095

2. Uma função para percorrer a árvore de decisão treinada e realizar a classificação de uma nova instância (do conjunto de teste);

In [4]:
def classify(tree, classes, input):
    """classify the input using the given decision tree"""

    # se este é um nó folha, retorna seu valor
    if tree in classes:
        return tree

    # caso contrário, encontre a subárvore correta
    attribute, subtree_dict = tree

    subtree_key = input.get(attribute)  # Nenhum se a entrada estiver faltando atributo

    if subtree_key not in subtree_dict: # se não houver subárvore para a chave
        subtree_key = None              # vamos usar a subárvore não

    subtree = subtree_dict[subtree_key] # escolha a subárvore apropriada
    return classify(subtree, classes, input) # e usá-lo para classificar a entrada


# obtendo as classes possiveis para a àrvore
classes = list(set([label for item, label in dataset if label]))

print('\n')
print('[INFO] Dados de amostras (ultimas duas linhas) para teste de resultado:')

print('Nublado / Quente / Normal / Falso ->', classify(tree, classes,
    { 'Tempo' : 'Nublado',
      'Temperatura' : 'Quente',
      'Umidade' : 'Normal',
      'Ventoso' : 'Falso'} ))
print('Chuvoso / Amena / Alta / Verdadeiro ->', classify(tree, classes,
    { 'Tempo' : 'Chuvoso',
      'Temperatura' : 'Amena',
      'Umidade' : 'Alta',
      'Ventoso' : 'Verdadeiro'} ))



[INFO] Dados de amostras (ultimas duas linhas) para teste de resultado:
Nublado / Quente / Normal / Falso -> Sim
Chuvoso / Amena / Alta / Verdadeiro -> Nao


3. O mecanismo de bootstrap (amostragem com reposição) para geração de subconjuntos a partir do conjunto de dados de treinamento originais. Cada bootstrap será utilizado para o treinamento de uma árvore no aprendizado ensemble

In [5]:
def bootstrap(dataset):
    bootstrap = dataset.sample(n = len(dataset), replace = True)
    return bootstrap

result_bootstrap = bootstrap(pd.read_csv('datasets/voting.csv'))

In [6]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

result_cross = cross_validation_split(result_bootstrap.values.tolist(), 10)

In [7]:
# TODO: DIVISÃO TREINO E TESTE
def train_split(dataset, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        return train_set

def test_split(dataset, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    for fold in folds:
        test_set = list(fold)
        return test_set


# Não está retendo os dados do banco de bootstrap, é os mesmos dados, mas ainda tá flutuando e gerando listas diferentes, temos que discutir se isso tem algum problema 


test_set = test_split(result_cross, 10) 
print('[INFO] definindo folds para teste')

print('\n[INFO] definindo folds para treino')
train_set = train_split(result_cross, 10)

[INFO] definindo folds para teste

[INFO] definindo folds para treino


In [8]:
def build_from_split(headers, dataset_from_bootstrap): 
    new_dataset = list()    
    for row in dataset_from_bootstrap[0:len(dataset_from_bootstrap)]:
        out_dic = {}
        for header_index in range(len(headers)):
            out_dic.update({ headers[header_index]: row[header_index]})
        
        # captura a classe
        class_key = list(out_dic.items())[-1]
        # remove a classe captura do dicionario
        del out_dic[list(out_dic.keys())[-1]]
        # montando o dataset de saída
        data_tuple = (out_dic, class_key[1])
        new_dataset.append(data_tuple)
    return new_dataset

# obtebdo os cabeçalhos
headers = result_bootstrap.columns.tolist()
#print(build_tree(build_from_bootstrap(headers, result_bootstrap.values.tolist())))


In [9]:
# construindo a árvore a partir do split vindo do cross
new_train = list()
for i in train_set: # treino
    for j in i:
        new_train.append(j)

new_tree_from_train = build_tree(build_from_split(headers, new_train))

# construindo a árvore a partir do split vindo do cross
new_test = list()
for i in test_set: # teste
    for j in i:
        new_test.append(j)
        
classes = list(set([label for item, label in build_from_split(headers, new_train) if label]))
    
classe_teste = list()
preditos_teste = list()
for current in build_from_split(headers, new_test):
    preditos_teste.append(current[1])
    classe_teste.append(classify(new_tree_from_train, classes, current[0]))
  
#print('\n[INFO] resultado real dos dados de teste: ')
#print(classe_teste)
#print('\n[INFO] resultado preditos esperado: ')
#print(preditos_teste)

[{'class': 'republican', 'amount': 144}, {'class': 'democrat', 'amount': 243}]
{'class': 'democrat', 'amount': 243}
[INFO] Atributo:  handicapped-infants - entropia média:  0.8078620700104473
[INFO] Atributo:  handicapped-infants - ganho:  0.14440355542621686
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.9520386044489699
[INFO] Atributo:  water-project-cost-sharing - ganho:  0.0002270209876942264
--
[INFO] Atributo:  adoption-of-the-budget-resolution - entropia média:  0.5066823350447823
[INFO] Atributo:  adoption-of-the-budget-resolution - ganho:  0.4455832903918818
--
[INFO] Atributo:  physician-fee-freeze - entropia média:  0.2510369976364611
[INFO] Atributo:  physician-fee-freeze - ganho:  0.7012286278002031
--
[INFO] Atributo:  el-salvador-aid - entropia média:  0.5217019559946126
[INFO] Atributo:  el-salvador-aid - ganho:  0.4305636694420516
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.8194142370839989
[INFO] Atributo:  religious-gr

[INFO] Atributo:  immigration - ganho:  0.00971667930924372
--
[INFO] Atributo:  synfuels-corporation-cutback - entropia média:  0.06349433357809509
[INFO] Atributo:  synfuels-corporation-cutback - ganho:  0.009839401273484316
--
[INFO] Atributo:  education-spending - entropia média:  0.07060029856365006
[INFO] Atributo:  education-spending - ganho:  0.002733436287929339
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.06964447950454407
[INFO] Atributo:  superfund-right-to-sue - ganho:  0.0036892553470353334
--
[INFO] Atributo:  crime - entropia média:  0.056224613418275234
[INFO] Atributo:  crime - ganho:  0.01710912143330417
--
[INFO] Atributo:  duty-free-exports - entropia média:  0.058417146520616706
[INFO] Atributo:  duty-free-exports - ganho:  0.014916588330962696
--
[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.06793205562934243
[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.005401679222236971
--
-------------

In [10]:
matrix = list()
accuracy = list()
recall = list()
precision = list()
score = list()

def classification_report(classe_teste, preditos_teste):
    true_class = 'democrat'
    negative_class = 'republican'
    # declarando a frêquencia
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    for (indice, v_real) in enumerate(classe_teste):
        v_predito = preditos_teste[indice]
        # se trata de um valor real da classe positiva
        if v_real == true_class:
            tp += 1 if v_predito == v_real else 0
            fp += 1 if v_predito != v_real else 0
        else:
            tn += 1 if v_predito == v_real else 0
            fn += 1 if v_predito != v_real else 0
                
    # calculando os valores de frêquencia
    accuracy.append((tp+tn)/(tp+fp+tn+fn))
    recall.append(tp/(tp+fn))
    precision.append(tp/(tp+fp))
    score.append(2*tp/(2*tp+fp+fn))
    
    # desenhando a matriz de confusão
    print(np.array([
        [ tp, fp ],
        [ fn, tn ]
    ]))
    
classification_report(classe_teste, preditos_teste)
print('accuracy: ', accuracy)
print('recall: ', recall)
print('precision: ', precision)

[[26  0]
 [ 0 17]]
accuracy:  [1.0]
recall:  [1.0]
precision:  [1.0]


In [11]:
print(classe_teste)
print('\n')
print(preditos_teste)

['republican', 'democrat', 'democrat', 'democrat', 'democrat', 'republican', 'republican', 'democrat', 'republican', 'republican', 'democrat', 'republican', 'democrat', 'democrat', 'democrat', 'democrat', 'republican', 'democrat', 'democrat', 'republican', 'democrat', 'democrat', 'democrat', 'democrat', 'democrat', 'democrat', 'democrat', 'republican', 'democrat', 'republican', 'democrat', 'democrat', 'republican', 'republican', 'republican', 'republican', 'republican', 'democrat', 'democrat', 'democrat', 'democrat', 'republican', 'republican']


['republican', 'democrat', 'democrat', 'democrat', 'democrat', 'republican', 'republican', 'democrat', 'republican', 'republican', 'democrat', 'republican', 'democrat', 'democrat', 'democrat', 'democrat', 'republican', 'democrat', 'democrat', 'republican', 'democrat', 'democrat', 'democrat', 'democrat', 'democrat', 'democrat', 'democrat', 'republican', 'democrat', 'republican', 'democrat', 'democrat', 'republican', 'republican', 'republican', 

Sugere-se utilizar k = 10 folds na validação cruzada. Destes k folds, k - 1 folds serão usados para construção do modelo de Florestas Aleatórias com ntree árvores, aplicando-se o algoritmo de indução de árvores de decisão combinado aos processos de amostragem de instâncias (bootstrap, i.e., amostragem com reposição) e de atributos (com base no parâmetro m). O fold restante (de teste) será usado para avaliar o desempenho do ensemble. Isto é, em cada etapa da validação cruzada, será gerada não apenas uma árvore de decisão, mas múltiplos modelos de árvore de decisão (tantos quantos forem o valor de ntree), os quais irão compor o modelo de Florestas Aleatórias. Perceba que como o treinamento de cada árvore envolve aleatoriedade, haverá diversidade entre as árvores geradas a partir do mesmo conjunto de k - 1 folds de treinamento. Este processo de treinamento e teste será repetido k vezes, de acordo com o funcionamento básico da estratégia de validação cruzada visto em aula. O uso de validação cruzada repetida é opcional.

In [12]:
# TODO: FLORESTAS ALEATÓRIAS

dados_originais = pd.read_csv('datasets/voting.csv')

n_trees = 10
my_tress = list()
my_cases_for_test = list()
for n in range(n_trees):
    n_bootstrap = bootstrap(dados_originais)
 
    # TODO: PRECISAMOS PEGAR A DIFERENÇA ENTRE DADOS ORIGINAIS PARA OBTER OS DADOS DE TESTE
    # diff_dataframe(n_bootstrap.values.tolist())
        
    n_cross = cross_validation_split(n_bootstrap.values.tolist(), 10)
    train_set = train_split(n_cross, 10)
    new_tree_from_train = build_tree(build_from_split(headers, new_train))
    my_tress.append(my_tress)

def forest_classify(trees, classes, input):
    votes = [classify(tree, classes, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]

[{'class': 'republican', 'amount': 144}, {'class': 'democrat', 'amount': 243}]
{'class': 'democrat', 'amount': 243}
[INFO] Atributo:  handicapped-infants - entropia média:  0.8078620700104473
[INFO] Atributo:  handicapped-infants - ganho:  0.14440355542621686
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.9520386044489699
[INFO] Atributo:  water-project-cost-sharing - ganho:  0.0002270209876942264
--
[INFO] Atributo:  adoption-of-the-budget-resolution - entropia média:  0.5066823350447823
[INFO] Atributo:  adoption-of-the-budget-resolution - ganho:  0.4455832903918818
--
[INFO] Atributo:  physician-fee-freeze - entropia média:  0.2510369976364611
[INFO] Atributo:  physician-fee-freeze - ganho:  0.7012286278002031
--
[INFO] Atributo:  el-salvador-aid - entropia média:  0.5217019559946126
[INFO] Atributo:  el-salvador-aid - ganho:  0.4305636694420516
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.8194142370839989
[INFO] Atributo:  religious-gr

[INFO] Atributo:  el-salvador-aid - entropia média:  0.07094975023764588
[INFO] Atributo:  el-salvador-aid - ganho:  0.002383984613933521
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.06575000633793057
[INFO] Atributo:  religious-groups-in-schools - ganho:  0.00758372851364883
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  0.0710185122191199
[INFO] Atributo:  anti-satellite-test-ban - ganho:  0.002315222632459507
--
[INFO] Atributo:  aid-to-nicaraguan-contras - entropia média:  0.07213435204455851
[INFO] Atributo:  aid-to-nicaraguan-contras - ganho:  0.0011993828070208934
--
[INFO] Atributo:  mx-missile - entropia média:  0.06956791743663204
[INFO] Atributo:  mx-missile - ganho:  0.003765817414947359
--
[INFO] Atributo:  immigration - entropia média:  0.06361705554233568
[INFO] Atributo:  immigration - ganho:  0.00971667930924372
--
[INFO] Atributo:  synfuels-corporation-cutback - entropia média:  0.06349433357809509
[INFO] Atributo:  synfuels-c

[INFO] Atributo:  anti-satellite-test-ban - ganho:  0.31127812445913283
--
[INFO] Atributo:  aid-to-nicaraguan-contras - entropia média:  0.0
[INFO] Atributo:  aid-to-nicaraguan-contras - ganho:  1.0
--
[INFO] Atributo:  immigration - entropia média:  0.0
[INFO] Atributo:  immigration - ganho:  1.0
--
[INFO] Atributo:  education-spending - entropia média:  1.0
[INFO] Atributo:  education-spending - ganho:  0.0
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.0
[INFO] Atributo:  superfund-right-to-sue - ganho:  1.0
--
[INFO] Atributo:  crime - entropia média:  1.0
[INFO] Atributo:  crime - ganho:  0.0
--
[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.0
[INFO] Atributo:  export-administration-act-south-africa - ganho:  1.0
--
----------------------------------------
[INFO] best_attribute:  el-salvador-aid
----------------------------------------
[{'class': 'democrat', 'amount': 2}]
{'class': 'democrat', 'amount': 2}
[{'class': 'republican', 

[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.7871107149038482
[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.07600985366278279
--
----------------------------------------
[INFO] best_attribute:  duty-free-exports
----------------------------------------
[{'class': 'republican', 'amount': 2}]
{'class': 'republican', 'amount': 2}
[{'class': 'democrat', 'amount': 5}]
{'class': 'democrat', 'amount': 5}
[{'class': 'democrat', 'amount': 1}]
{'class': 'democrat', 'amount': 1}
[{'class': 'democrat', 'amount': 2}]
{'class': 'democrat', 'amount': 2}
[{'class': 'republican', 'amount': 5}, {'class': 'democrat', 'amount': 5}]
{'class': 'republican', 'amount': 5}
[INFO] Atributo:  handicapped-infants - entropia média:  0.36096404744368116
[INFO] Atributo:  handicapped-infants - ganho:  0.6390359525563188
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.0
[INFO] Atributo:  water-project-cost-sharing - ganho:  1.0
--
[INFO] Atr

{'class': 'republican', 'amount': 9}
[INFO] Atributo:  handicapped-infants - entropia média:  0.4529325012980812
[INFO] Atributo:  handicapped-infants - ganho:  0.01606309229120001
--
[INFO] Atributo:  adoption-of-the-budget-resolution - entropia média:  0.4348515545596771
[INFO] Atributo:  adoption-of-the-budget-resolution - ganho:  0.03414403902960411
--
[INFO] Atributo:  el-salvador-aid - entropia média:  0.4689955935892812
[INFO] Atributo:  el-salvador-aid - ganho:  0.0
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.4689955935892812
[INFO] Atributo:  religious-groups-in-schools - ganho:  0.0
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  0.4348515545596771
[INFO] Atributo:  anti-satellite-test-ban - ganho:  0.03414403902960411
--
[INFO] Atributo:  aid-to-nicaraguan-contras - entropia média:  0.4689955935892812
[INFO] Atributo:  aid-to-nicaraguan-contras - ganho:  0.0
--
[INFO] Atributo:  mx-missile - entropia média:  0.4689955935892812
[INFO]

...

[INFO] Atributo:  crime - entropia média:  1.0
[INFO] Atributo:  crime - ganho:  0.0
--
[INFO] Atributo:  duty-free-exports - entropia média:  1.0
[INFO] Atributo:  duty-free-exports - ganho:  0.0
--
----------------------------------------
[INFO] best_attribute:  handicapped-infants
----------------------------------------
[{'class': 'democrat', 'amount': 1}]
{'class': 'democrat', 'amount': 1}
[{'class': 'republican', 'amount': 1}]
{'class': 'republican', 'amount': 1}
[{'class': 'republican', 'amount': 4}]
{'class': 'republican', 'amount': 4}
[{'class': 'republican', 'amount': 7}]
{'class': 'republican', 'amount': 7}
[{'class': 'republican', 'amount': 2}, {'class': 'democrat', 'amount': 223}]
{'class': 'democrat', 'amount': 223}
[INFO] Atributo:  handicapped-infants - entropia média:  0.06810535760576691
[INFO] Atributo:  handicapped-infants - ganho:  0.005228377245812488
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.0633704258025824
[INFO] Atributo:  water-proj

[INFO] Atributo:  education-spending - entropia média:  0.8565821749375165
[INFO] Atributo:  education-spending - ganho:  0.1179072284605358
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.932809776809723
[INFO] Atributo:  superfund-right-to-sue - ganho:  0.041679626588329355
--
[INFO] Atributo:  crime - entropia média:  0.9744677414273976
[INFO] Atributo:  crime - ganho:  2.1661970654784923e-05
--
[INFO] Atributo:  duty-free-exports - entropia média:  0.8422357860749325
[INFO] Atributo:  duty-free-exports - ganho:  0.1322536173231198
--
[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.9190821749375164
[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.05540722846053592
--
----------------------------------------
[INFO] best_attribute:  water-project-cost-sharing
----------------------------------------
[{'class': 'republican', 'amount': 10}, {'class': 'democrat', 'amount': 12}]
{'class': 'democrat', 'amount': 12}
[INFO] A

[INFO] Atributo:  crime - ganho:  0.3319412415769716
--
[INFO] Atributo:  duty-free-exports - entropia média:  0.7053988133042884
[INFO] Atributo:  duty-free-exports - ganho:  0.2468668121323757
--
[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.8197268177739422
[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.13253880766272197
--
----------------------------------------
[INFO] best_attribute:  physician-fee-freeze
----------------------------------------
[{'class': 'republican', 'amount': 137}, {'class': 'democrat', 'amount': 15}]
{'class': 'republican', 'amount': 137}
[INFO] Atributo:  handicapped-infants - entropia média:  0.45833170397095785
[INFO] Atributo:  handicapped-infants - ganho:  0.006479004869207106
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.4267018217408023
[INFO] Atributo:  water-project-cost-sharing - ganho:  0.038108887099362665
--
[INFO] Atributo:  adoption-of-the-budget-resolution - entropia

[INFO] Atributo:  immigration - entropia média:  0.32451124978365314
[INFO] Atributo:  immigration - ganho:  0.14448434380562808
--
[INFO] Atributo:  education-spending - entropia média:  0.4689955935892812
[INFO] Atributo:  education-spending - ganho:  0.0
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.4689955935892812
[INFO] Atributo:  superfund-right-to-sue - ganho:  0.0
--
[INFO] Atributo:  crime - entropia média:  0.4689955935892812
[INFO] Atributo:  crime - ganho:  0.0
--
[INFO] Atributo:  duty-free-exports - entropia média:  0.4689955935892812
[INFO] Atributo:  duty-free-exports - ganho:  0.0
--
[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.2
[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.2689955935892812
--
----------------------------------------
[INFO] best_attribute:  export-administration-act-south-africa
----------------------------------------
[{'class': 'republican', 'amount': 4}]
{'class': 'republic

[INFO] Atributo:  superfund-right-to-sue - ganho:  0.04104670581922587
--
[INFO] Atributo:  crime - entropia média:  0.9759479941023754
[INFO] Atributo:  crime - ganho:  0.018082217374581044
--
[INFO] Atributo:  duty-free-exports - entropia média:  0.8619098542533496
[INFO] Atributo:  duty-free-exports - ganho:  0.13212035722360682
--
[INFO] Atributo:  export-administration-act-south-africa - entropia média:  0.6891481572150076
[INFO] Atributo:  export-administration-act-south-africa - ganho:  0.3048820542619489
--
----------------------------------------
[INFO] best_attribute:  export-administration-act-south-africa
----------------------------------------
[{'class': 'republican', 'amount': 4}, {'class': 'democrat', 'amount': 3}]
{'class': 'republican', 'amount': 4}
[INFO] Atributo:  handicapped-infants - entropia média:  0.9649839288804956
[INFO] Atributo:  handicapped-infants - ganho:  0.020244207153756077
--
[INFO] Atributo:  adoption-of-the-budget-resolution - entropia média:  0.0

4. O mecanismo de amostragem de m atributos a cada divisão de nó, a partir dos quais será selecionado o melhor atributo de acordo com o critério de Ganho de Informação

5. O treinamento de um ensemble de árvores de decisão, adotando os mecanismos de bootstrap e seleção de atributos com amostragem, como mencionados acima


6. O mecanismo de votação majoritária entre as múltiplas árvores de decisão no ensemble, para classificação de novas instâncias utilizando o modelo de Florestas Aleatórias

7. A técnica de validação cruzada (cross-validation) estratificada, para avaliar poder de generalização do modelo e a variação de desempenho de acordo com diferentes valores para os parâmetros do algoritmo (ex., número de árvores no ensemble)

8. Avaliação do impacto do número de árvores no desempenho do ensemble

In [13]:
# TODO:
def view_forest():
        g = Digraph('quero dormir!')
        g.graph_attr.update(fontsize='40')
        g.node_attr.update(style='filled', fontname='Arial')
        g.edge_attr.update(fontname='Arial')

        i = 0
        for tree in new_tree_from_train:
            i += 1
            with g.subgraph(name='cluster_' + str(i)) as c:
                nodes = [tree.root]
                while len(nodes) != 0:
                    node = nodes.pop(0)
                    c.node(str(id(node)), str(node), shape="box" if node.is_leaf else "ellipse", color="gray" if node.is_leaf else "green3")
                    for value, child in node.children.items():
                        c.edge(str(id(node)), str(id(child)), label=str(value))
                        nodes.append(child)
                c.attr(label='Tree ' + str(i))
        g.view()